# Projeto de análise de dados sobre os registro de batalhas dos 1000 melhores jogadores em desafio 1v1 no clash royale, **Dado retirado em novembro de 2018**

### ⛏ Importação, tratamento, formatação básica dos dados e funções básicas

In [ ]:
# IMPORTANDO AS BIBLIOTECAS USADAS NO DESENVOLVIMENTO DAS ANÁLISES
import pandas as pd
from collections import Counter
from itertools import combinations
import os
import sys
# PARA QUE O NOTEBOOK CONSIGA ACESSAR A PASTA SRC
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
# IMPORTANDO AS FUNÇÕES PARA AUXILIAR NA ANÁLISE DOS DADOS, FUNÇÕES CONTIDAS DENTRO DA PASTA SRC/FEATURES
from src.features.criar_funcs import func_verificar_vencedor, contar_cartas_por_linhas, trasnformar_series_em_df, retorna_deck_venc_ou_ambos
from src.utils.criar_funcs_utils import criar_dataframe_juntando_series

### TRATAMENTO E FORMATAÇÃO DE DADOS

In [ ]:
# IMPORTANDO A BASE DE DADOS
df_clash = pd.read_excel('../data/raw/Clash Royale Top Battles v1.xlsx')

In [ ]:
# EXCLUINDO A COLUNA Unnamed: 0 QUE VEM NA BASE DE DADOS
del df_clash['Unnamed: 0']

In [ ]:
# CRIAR UMA CÓPIA DO DF_CLASH, PARA UTILIZAR SEM DANIFICAR O DF ORIGINAL
df_clash_copia = df_clash.copy()

In [ ]:
# CRIAR A COLUNA COM OS DECKS VENCEDORES, AINDA NULO PARA QUE SE POSSA FAZER UMA ANÁLISE POSTERIORMENTE E ADICIONAR VALOR 1 OU 2 PARA SABER O VENCEDOR DE CADA PARTIDA
df_clash_copia['Deck Vencedor'] = None

In [ ]:
# FUNÇÃO PARA CRIAR A COLUNA, DIZENDO QUAL DECK FOI O VENCEDOR
def deck_vencedor(valor):
    if valor['p1_crowns'] > valor['p2_crowns']:
        df_clash_copia.loc[valor.name, 'Deck Vencedor'] = 1
    else:
        df_clash_copia.loc[valor.name, 'Deck Vencedor'] = 2

df_clash_copia.apply(deck_vencedor, axis=1)

### 📍 Extrair o as cartas mais utilizadas nos decks

In [ ]:
# CHAMA A FUNÇÃO contar_cartas_por_linhas, ONDE NELA SE RETORNA UM DICIONÁRIO COM TODOS OS 2 DECKS DE CADA PARTIDA
# ATRIBUI ESSES VALORES A o DICIONÁRIO lista_cartas_por_batalhalinha
lista_cartas_por_batalhalinha = retorna_deck_venc_ou_ambos(2, df_clash_copia, 1)

# CRIANDO UM Counter PARA CONTAR A QUANTIDADE DE APARIÇÕES DE CADA CARTA, EM CADA PARTIDA
dic_aparicoes_cartas = Counter()
for i, x in lista_cartas_por_batalhalinha.items():
   # UM FOR DENTRO DO DICIONÁRIO PEGANDO AMBOS OS DECKS E USANDO O COUNTER PARA CONTAR A QUANTIDADE DE VEZES QUE CADA CARTA FOI UTILIZADA (APARIÇÕES DAS CARTAS NAS PARTIDAS)
   dic_aparicoes_cartas.update(x)

# CRIANDO UM DATAFRAME, MOSTRANDO A QUANTIDADE DE APARIÇÕES POR CARTA
df_cartas_mais_usadas = trasnformar_series_em_df(dic_aparicoes_cartas, 'Quantidade')

df_cartas_mais_usadas

### 📍 Extrair a taxa de vitória por carta (quantas vezes ela aparece nos decks vitoriosos)

In [ ]:
# CHAMA A FUNÇÃO func_verificar_vencedor QUE RETORNA UM DICIONÁRIO COM APENAS OS DECKS VENCEDORES POR PARTIDAS
# ATRIBUI ESSES VALORES A O DICIONÁRIO lista_cartas_mais_vitoriosas
lista_cartas_mais_vitoriosas = retorna_deck_venc_ou_ambos(1, df_clash_copia, 1)

# CRIANDO UM Counter PARA CONTAR A QUANTIDADE DE APARIÇÕES DAS CARTAS NOS DECKS VITORIOSOS
dic_aparicoes_cartas_mais_vitoriosas = Counter()
for i, x in lista_cartas_mais_vitoriosas.items():
    # UM FOR DENTRO DO DICIONÁRIO PEGANDO AS CARTAS USADAS NOS DECKS VENCEDORES E UTILIZANDO O COUNTER PARA CONTAR AS CARTAS QUE APARECEM NO DECK 
    dic_aparicoes_cartas_mais_vitoriosas.update(x)

# # CRIANDO UM DATAFRAME, MOSTRANDO A QUANTIDADE DE APARIÇÕES POR CARTA NOS DECKS VENCEDORES
df_cartas_mais_vitoriosas = trasnformar_series_em_df(dic_aparicoes_cartas_mais_vitoriosas, 'Cartas mais Vencedoras')
df_cartas_mais_vitoriosas

In [ ]:
# CRIANDO UM DATAFRAME JUNTANDO AMBOS DATAFRAME, COM AS CARTAS MAIS UTILIZADAS/APARIÇÕES DAS CARTAS EM TODAS AS PARTIDAS E AS CARTAS QUE MAIS APARECEM  NOS DECKS VENCEDORES
df_taxa_vitorias = criar_dataframe_juntando_series(df_cartas_mais_usadas, df_cartas_mais_vitoriosas)

# FUNÇÃO PARA RETORNAR A PORCENTAGEM ENTRE A DIVISÃO DAS CARTAS MAIS VENCEDORES POR A QUANTIDADE DE VEZES QUE ESSA CARTA APARECEU EM AMBOS OS DECK PERDENDO OU GANHANDO
def tirar_taxa_vitoria(valor):
    return valor['Cartas mais Vencedoras'] / valor['Quantidade'] * 100

# CRIAR O DATAFRAME FORMATADO COM A TAXA DE VITÓRIA DE CADA CARTA
df_taxa_vitorias = pd.DataFrame(df_taxa_vitorias.apply(tirar_taxa_vitoria, axis=1),columns=['Taxa de Vitória'])
df_taxa_vitorias['Taxa de Vitória'] = df_taxa_vitorias['Taxa de Vitória'].map('{:.2f}'.format)
df_taxa_vitorias.sort_values(by='Taxa de Vitória',ascending=False, inplace=True)
df_taxa_vitorias

### 📍 Extrair combinações de cartas mais utilizadas

In [ ]:
# RETORNAR O DICIONARIO COM OS DECKS VITORIOSOS
dic_listas_combinacoes_vitoriosas = retorna_deck_venc_ou_ambos(1, df_clash_copia,1)

In [ ]:
# CRIAR UM COUNTER PARA CONTAR OS COMBOS USADOS NOS DECKS VENCEDORES NAS PARTIDAS
contagem_combinacoes_vitoriosas = Counter()
for i, deck in dic_listas_combinacoes_vitoriosas.items():
    # um for passando o deck vencedor, com isso é utilizado a biblioteca combinations que retorna todas as combinações possíveis do deck
    comb = tuple(combinations(deck, 3))
    # criar um tuple com uma ordem padrão, para que não haja combos duplicados, ex: (1,2,3) e (2,3,1)
    # assim ele organiza sempre na mesma ordem (1,2,3) para que counter sempre possa contar corretamente sem contar o mesmo item como diferentes
    remover_duplicata = [tuple(sorted(c)) for c in comb]
    # uso o counter para contar a quantidade de vezes que os combos apareceram, passando a lista de combinações possíveis
    contagem_combinacoes_vitoriosas.update(remover_duplicata)


# cria uma series filtrando somente os combos com aparições maior que 100, isso indica que é um combo real e não só uma combinação aleatória
series_combinacoes = pd.Series(contagem_combinacoes_vitoriosas).sort_values(ascending=False)
mask_formar_combos = series_combinacoes > 100
series_combinacoes  = series_combinacoes[mask_formar_combos]


# cria o dataframe com a quantidade de aparições dos combos nas partidas
df_qtd_combos_vencedores = trasnformar_series_em_df(series_combinacoes, 'Combos mais Vencedores')
df_qtd_combos_vencedores.index = df_qtd_combos_vencedores.index.map(lambda x: '+'.join(x))

df_qtd_combos_vencedores

In [ ]:
# RETORNA O DICIONÁRIO COM OS DECKS
dic_listas_combinacoes_mais_usadas = retorna_deck_venc_ou_ambos(2, df_clash_copia, 1)

In [ ]:
# CRIAR O DATAFRAME, QUE CONTÉM OS COMBOS MAIS USADOS NAS PARTIDAS
contagem_combinacoes_mais_usadas = Counter()
for i, deck_m_usados in dic_listas_combinacoes_mais_usadas.items():
    # um for passando por ambos os decks, com isso é utilizado a biblioteca combinations que retorna todas as combinações possíveis dos decks
    comb_mais_usadas = list(combinations(deck_m_usados, 3))
    # criar um tuple com uma ordem padrão, para que não haja combos duplicados, ex: (1,2,3) e (2,3,1)
    # assim ele organiza sempre na mesma ordem (1,2,3) para que counter sempre possa contar corretamente sem contar o mesmo item como diferentes
    remover_duplicata = [tuple(sorted(c)) for c in comb_mais_usadas]
    # uso o counter para contar a quantidade de vezes que os combos apareceram, passando a lista de combinações possíveis
    contagem_combinacoes_mais_usadas.update(remover_duplicata)

# cria uma series filtrando somente os combos com aparições maior que 100, isso indica que é um combo real e não só uma combinação aleatória
series_combinacoes_mais_usadas = pd.Series(contagem_combinacoes_mais_usadas).sort_values(ascending=False)
mask_formar_combos_mais_usadas = series_combinacoes_mais_usadas > 100
series_combinacoes_mais_usadas = series_combinacoes_mais_usadas[mask_formar_combos_mais_usadas]

# cria o dataframe com a quantidade de aparições dos combos nas partidas
df_qtd_combos_mais_usados = pd.DataFrame(series_combinacoes_mais_usadas, columns=['Combos mais Utilizados'])
df_qtd_combos_mais_usados.index = df_qtd_combos_mais_usados.index.map(lambda x: '+'.join(x))
df_qtd_combos_mais_usados

In [ ]:
# CRIANDO UM DATAFRAME SOMANDO OS 2 DF COM A QUANTIDADE DE COMBOS UTILIZADOS NOS DECKS VITORIOSOS E EM AMBOS OS DECKS USADOS
df_taxa_combos = criar_dataframe_juntando_series(df_qtd_combos_vencedores, df_qtd_combos_mais_usados)
# EXCLUI QUALQUER LINHA EM BRANCO OU NONE
df_taxa_combos.dropna(inplace=True)

# FUNÇÃO QUE RETORNA A PORCENTAGEM COM A DIVISÃO ENTRE A QUANTIDADE DE COMBOS VENCEDORES E AS APARIÇÕES TOTAIS
def tirar_taxa_combos(valor):
    return valor['Combos mais Vencedores'] / valor['Combos mais Utilizados'] * 100

grupo = df_taxa_combos.apply(tirar_taxa_combos, axis=1)
df_taxa_combos = pd.DataFrame(grupo, columns=['Taxa de Vitória por Combos']).sort_values(by='Taxa de Vitória por Combos', ascending=False)
df_taxa_combos['Taxa de Vitória por Combos'] = df_taxa_combos['Taxa de Vitória por Combos'].map('{:.2f}'.format)
df_taxa_combos

### 📍 Extrair a média de coroas entre os vencedores e os pededores

In [31]:
# RETORNA A DIFERENÇA ENTRE AS COROAS POR PARTIDAS
def tirar_media_coroas(valor):
    if valor['p1_crowns'] > valor['p2_crowns']:
        return valor['p1_crowns'] - valor['p2_crowns']
    else:
        return valor['p2_crowns'] - valor['p1_crowns']

# CONTA A QUANTIDADE DE VEZES QUE CADA DIFERENÇÃ DE PARTIDA ACONTECE
grupo = df_clash_copia[['p1_crowns', 'p2_crowns']].apply(tirar_media_coroas, axis=1)

# RETORNA A PORCENTAGEM DE VITÓRIAS POR DETERMINADA DIFERENÇA DE COROAS
df_taxa_dif_coras = grupo.value_counts() / df_clash_copia.shape[0] * 100
df_taxa_dif_coras = pd.DataFrame(df_taxa_dif_coras)
df_taxa_dif_coras['count'] = df_taxa_dif_coras['count'].map('{:.2f}'.format)
df_taxa_dif_coras.rename(columns={
    'count': 'Taxa diferença coroas por Parida'
}, inplace=True
)

### 🛠 Gerar os arquivos formatados e processados

In [32]:
# RETORNA CADA DATAFRAME PROCESSEADO PARA UM PKL DENTRO DA PASTA PROCESSED 
df_clash_copia.to_pickle('../data/processed/df_clash_copia.pkl')
df_cartas_mais_usadas.to_pickle('../data/processed/df_cartas_mais_usadas.pkl')
df_taxa_vitorias.to_pickle('../data/processed/df_taxa_vitorias.pkl')
df_taxa_combos.to_pickle('../data/processed/df_taxa_combos.pkl')
df_taxa_dif_coras.to_pickle('../data/processed/df_taxa_dif_coroas.pkl')